In [1]:
'''
Notebook for FIG4 - Compare the case of the Punisher circuit with two toggle switches, two toggle switches with essential gene co-expression and two toggle switches only
'''
# By Kirill Sechkar

# PACKAGE IMPORTS 
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio, transform as bktransform
from bokeh.colors import RGB as bkRGB
from math import pi
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS
import synthetic_circuits as circuits
from cell_model import *

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [2]:
# TWO TOGGLE SWITCHES AND THE PUNISHER
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het,\
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.twotoggles_punisher_initialise,
    circuits.twotoggles_punisher_ode,
    circuits.twotoggles_punisher_F_calc,
    circuits.twotoggles_punisher_eff_m_het_div_k_het,
    par, init_conds)  # load the circuit

In [3]:
# TWO TOGGLE SWITCHES AND THE PUNISHER
# PARAMETERISE THE CIRCUIT (DEFAULT)

 # TOGGLE SWITCHES
for togswitchnum in (1, 2):  # cycle through toggle switches
    for toggenenum in (1, 2):  # cycle through the genes of the current switch
        par['c_tog' + str(togswitchnum) + str(toggenenum)] = 1  # copy no. (nM)
        par['a_tog' + str(togswitchnum) + str(toggenenum)] = 1e5/2 # promoter strength (unitless)

        # transcription regulation function
        reg_func_string = 'dna(tog' + str(togswitchnum) + str(toggenenum) + '):p_tog' + str(togswitchnum) + str(
            (toggenenum - 2) % 2 + 1)  # dna(rep1):p_rep3, dna(rep2):p_rep1 or dna(rep3):p_rep2
        par['K_' + reg_func_string] = 2500  # half-saturation constant
        par['eta_' + reg_func_string] = 2  # Hill coefficient
        par['baseline_tog' + str(togswitchnum) + str(
            toggenenum)] = 0.025  # baseline transcription activation due to leakiness
        par['p_tog' + str(togswitchnum) + str(
            toggenenum) + '_ac_frac'] = 1  # active fraction of protein (i.e. share of molecules not bound by the inducer)
    # break symmetry for each of the toggle switches
    init_conds['m_tog' + str(togswitchnum) + '1'] = 4000

# PUNISHER
# switch gene conc
par['c_switch'] = 10.0  # gene concentration (nM)
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch']=0.01836 # rate of protein degradation per protease molecule (1/nM/h)
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
init_conds['cat_pb'] = 10.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
par['a_cat'] = 500.0  # promoter strength (unitless)
# synthetic protease gene
par['c_prot'] = 10.0  # gene concentration (nM)
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['p_switch_ac_frac'] = 0.87  # active fraction of protein (i.e. share of molecules bound by the inducer)

# CULTURE MEDIUM
init_conds['s'] = 0.5   # nutrient quality (unitless)
par['h_ext'] = 10.5 * (10.0 ** 3)   # extracellular chloramphenicol concentration (nM)

# save parameters and initial conditions
par_pun = par.copy()
init_conds_pun = init_conds.copy()

In [4]:
# SET SIMULATION PARAMETERS
# diffrax simulator
savetimestep = 1  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver
# simulation time frames
tf = (0,50) # time frame for simulation before burdensome gene loss

In [5]:
# TWO TOGGLE SWITCHES AND THE PUNISHER
# GET STEADY STATE GROWTH RATES FOR DIFFERENT MUTATIONS OF THE TOGGLES

# all toggles functional
#simulate
par_nomut=par.copy()
sol = ode_sim(par_nomut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_nomut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_nomut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_pun_nomut=np.float64(ls[-1])

# half of a toggle (tog11 gene) mutated
#simulate
par_halfmut=par.copy()
par_halfmut['func_tog11']=0
sol = ode_sim(par_halfmut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_halfmut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_halfmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_pun_halfmut=np.float64(ls[-1])

# two halves of a toggle (tog11 and tog21 genes) mutated
#simulate
par_twohalfmut=par.copy()
par_twohalfmut['func_tog11']=0
par_twohalfmut['func_tog21']=0
sol = ode_sim(par_twohalfmut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_twohalfmut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_twohalfmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_pun_twohalfmut=np.float64(ls[-1])

# one toggle (tog11 and tog12 genes) mutated
par_onemut=par.copy()
par_onemut['func_tog11']=0
par_onemut['func_tog12']=0
sol = ode_sim(par_onemut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_onemut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_onemut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_pun_onemut=np.float64(ls[-1])

# one toggle and a half (tog11, tog12 and tog21 genes) mutated
par_oneandhalfmut=par.copy()
par_oneandhalfmut['func_tog11']=0
par_oneandhalfmut['func_tog12']=0
par_oneandhalfmut['func_tog21']=0
sol = ode_sim(par_oneandhalfmut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_oneandhalfmut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_oneandhalfmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_pun_oneandhalfmut=np.float64(ls[-1])

# both toggles (all toggle genes) mutated
par_allmut=par.copy()
par_allmut['func_tog11']=0
par_allmut['func_tog12']=0
par_allmut['func_tog21']=0
par_allmut['func_tog22']=0
sol = ode_sim(par_allmut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_allmut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_allmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_pun_allmut=np.float64(ls[-1])            

In [6]:
# TWO TOGGLE SWITCHES ONLY
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het,\
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.twotoggles_only_initialise,
    circuits.twotoggles_only_ode,
    circuits.twotoggles_only_F_calc,
    circuits.twotoggles_only_eff_m_het_div_k_het,
    par, init_conds)  # load the circuit

In [7]:
# TWO TOGGLE SWITCHES ONLY
# PARAMETERISE THE CIRCUIT - SAME VALUES AS TOGGLE SWITCHES WITH THE PUNISHER

 # TOGGLE SWITCHES
for togswitchnum in (1, 2):  # cycle through toggle switches
    for toggenenum in (1, 2):  # cycle through the genes of the current switch
        par['c_tog' + str(togswitchnum) + str(toggenenum)] = par_pun['c_tog' + str(togswitchnum) + str(toggenenum)]  # copy no. (nM)
        par['a_tog' + str(togswitchnum) + str(toggenenum)] = par_pun['a_tog' + str(togswitchnum) + str(toggenenum)]  # promoter strength (unitless)

        # transcription regulation function
        reg_func_string = 'dna(tog' + str(togswitchnum) + str(toggenenum) + '):p_tog' + str(togswitchnum) + str(
            (toggenenum - 2) % 2 + 1)  # dna(rep1):p_rep3, dna(rep2):p_rep1 or dna(rep3):p_rep2
        par['K_' + reg_func_string] = par_pun['K_' + reg_func_string]  # half-saturation constant
        par['eta_' + reg_func_string] = par_pun['eta_' + reg_func_string]  # Hill coefficient
        par['baseline_tog' + str(togswitchnum) + str(
            toggenenum)] = par_pun['baseline_tog' + str(togswitchnum) + str(toggenenum)]  # baseline transcription activation due to leakiness
        par['p_tog' + str(togswitchnum) + str(
            toggenenum) + '_ac_frac'] = par_pun['p_tog' + str(togswitchnum) + str(toggenenum) + '_ac_frac']  # active fraction of protein (i.e. share of molecules not bound by the inducer)
    # break symmetry for each of the toggle switches
    init_conds['m_tog' + str(togswitchnum) + '1'] = init_conds_pun['m_tog' + str(togswitchnum) + '1']

# NO PUNISHER CIRCUIT OR CHLORAMPHENICOL IN THE MEDIUM
init_conds['h'] = 0.0  # no chloramphenicol in the medium

In [8]:
# TWO TOGGLE SWITCHES ONLY
# GET STEADY STATE GROWTH RATES FOR DIFFERENT MUTATIONS OF THE TOGGLES

# all toggles functional
#simulate
par_nomut=par.copy()
sol = ode_sim(par_nomut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_nomut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_nomut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_only_nomut=np.float64(ls[-1])

# half of a toggle (tog11 gene) mutated
#simulate
par_halfmut=par.copy()
par_halfmut['func_tog11']=0
sol = ode_sim(par_halfmut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_halfmut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_halfmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_only_halfmut=np.float64(ls[-1])

# two halves of a toggle (tog11 and tog21 genes) mutated
#simulate
par_twohalfmut=par.copy()
par_twohalfmut['func_tog11']=0
par_twohalfmut['func_tog21']=0
sol = ode_sim(par_twohalfmut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_twohalfmut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_twohalfmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_only_twohalfmut=np.float64(ls[-1])

# one toggle (tog11 and tog12 genes) mutated
par_onemut=par.copy()
par_onemut['func_tog11']=0
par_onemut['func_tog12']=0
sol = ode_sim(par_onemut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_onemut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_onemut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_only_onemut=np.float64(ls[-1])

# one toggle and a half (tog11, tog12 and tog21 genes) mutated
par_oneandhalfmut=par.copy()
par_oneandhalfmut['func_tog11']=0
par_oneandhalfmut['func_tog12']=0
par_oneandhalfmut['func_tog21']=0
sol = ode_sim(par_oneandhalfmut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_oneandhalfmut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_oneandhalfmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_only_oneandhalfmut=np.float64(ls[-1])

# both toggles (all toggle genes) mutated
par_allmut=par.copy()
par_allmut['func_tog11']=0
par_allmut['func_tog12']=0
par_allmut['func_tog21']=0
par_allmut['func_tog22']=0
sol = ode_sim(par_allmut,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_allmut, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# record growth rate
_, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_allmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)
l_only_allmut=np.float64(ls[-1])            

In [9]:
# TWO TOGGLE SWITCHES WITH SYNTHETIC ADDICTION
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het,\
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.twotoggles_add_initialise,
    circuits.twotoggles_add_ode,
    circuits.twotoggles_add_F_calc,
    circuits.twotoggles_add_eff_m_het_div_k_het,
    par, init_conds)  # load the circuit

In [10]:
# TWO TOGGLE SWITCHES WITH SYNTHETIC ADDICTION
# PARAMETERISE THE CIRCUIT - SAME VALUES AS TOGGLE SWITCHES WITH THE PUNISHER

 # TOGGLE SWITCHES
for togswitchnum in (1, 2):  # cycle through toggle switches
    for toggenenum in (1, 2):  # cycle through the genes of the current switch
        par['c_tog' + str(togswitchnum) + str(toggenenum)] = par_pun['c_tog' + str(togswitchnum) + str(toggenenum)]  # copy no. (nM)
        par['a_tog' + str(togswitchnum) + str(toggenenum)] = par_pun['a_tog' + str(togswitchnum) + str(toggenenum)]  # promoter strength (unitless)
    
        # transcription regulation function
        reg_func_string = 'dna(tog' + str(togswitchnum) + str(toggenenum) + '):p_tog' + str(togswitchnum) + str(
            (toggenenum - 2) % 2 + 1)  # dna(rep1):p_rep3, dna(rep2):p_rep1 or dna(rep3):p_rep2
        par['K_' + reg_func_string] = par_pun['K_' + reg_func_string]  # half-saturation constant
        par['eta_' + reg_func_string] = par_pun['eta_' + reg_func_string]  # Hill coefficient
        par['baseline_tog' + str(togswitchnum) + str(
            toggenenum)] = par_pun['baseline_tog' + str(togswitchnum) + str(toggenenum)]  # baseline transcription activation due to leakiness
        par['p_tog' + str(togswitchnum) + str(
            toggenenum) + '_ac_frac'] = par_pun['p_tog' + str(togswitchnum) + str(toggenenum) + '_ac_frac']  # active fraction of protein (i.e. share of molecules not bound by the inducer)
    # break symmetry for each of the toggle switches
    init_conds['m_tog' + str(togswitchnum) + '1'] = init_conds_pun['m_tog' + str(togswitchnum) + '1']

# CO-EXPRESSED CAT (default values)
for togswitchnum in (1, 2):  # cycle through toggle switches
    for toggenenum in (1, 2):  # cycle through the genes of the current switch
        par['k+_cat' + str(togswitchnum) + str(toggenenum)] = 60  # ribosome binding rate (/h/nM)
        par['k-_cat' + str(togswitchnum) + str(toggenenum)] = 60  # ribosome unbinding rate (/h)
        
# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)

In [11]:
# TWO TOGGLE SWITCHES WITH SYNTHETIC ADDICTION
#  DEFINE RBS STRENTHS TO LOOK AT
k_plus_cats = np.logspace(np.log10(60/250), np.log10(60), 50)  # ribosome binding rates (/h/nM) to look at

In [12]:
ls_add_nomut = np.zeros_like(k_plus_cats)
ls_add_halfmut = np.zeros_like(k_plus_cats)
ls_add_twohalfmut = np.zeros_like(k_plus_cats)
ls_add_onemut = np.zeros_like(k_plus_cats)
ls_add_oneandhalfmut = np.zeros_like(k_plus_cats)
ls_add_allmut = np.zeros_like(k_plus_cats)

for kplus_cntr in range(0,len(k_plus_cats)):
    # set the rbs strengths
    for togswitchnum in (1, 2):  # cycle through toggle switches
        for toggenenum in (1, 2):  # cycle through the genes of the current switch
            par['k+_cat' + str(togswitchnum) + str(toggenenum)] = k_plus_cats[kplus_cntr]  # ribosome binding rate (/h/nM)
    
    # all toggles functional
    #simulate
    par_nomut=par.copy()
    sol = ode_sim(par_nomut,  # dictionary with model parameters
                  ode_with_circuit,  # ODE function for the cell with synthetic circuit
                  cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
                  # initial condition VECTOR
                  len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                  cellmodel_auxil.synth_gene_params_for_jax(par_nomut, circuit_genes),
                  # synthetic gene parameters for calculating k values
                  tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
                  rtol,
                  atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
    ts = np.array(sol.ts)
    xs = np.array(sol.ys)
    # record growth rate
    _, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_nomut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                           circuit_eff_m_het_div_k_het)
    ls_add_nomut[kplus_cntr]=np.float64(ls[-1])
    
    # half of a toggle (tog11 gene) mutated
    #simulate
    par_halfmut=par.copy()
    par_halfmut['func_tog11']=0
    sol = ode_sim(par_halfmut,  # dictionary with model parameters
                  ode_with_circuit,  # ODE function for the cell with synthetic circuit
                  cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
                  # initial condition VECTOR
                  len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                  cellmodel_auxil.synth_gene_params_for_jax(par_halfmut, circuit_genes),
                  # synthetic gene parameters for calculating k values
                  tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
                  rtol,
                  atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
    ts = np.array(sol.ts)
    xs = np.array(sol.ys)
    # record growth rate
    _, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_halfmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                           circuit_eff_m_het_div_k_het)
    ls_add_halfmut[kplus_cntr]=np.float64(ls[-1])
    
    # two halves of a toggle (tog11 and tog21 genes) mutated
    #simulate
    par_twohalfmut=par.copy()
    par_twohalfmut['func_tog11']=0
    par_twohalfmut['func_tog21']=0
    sol = ode_sim(par_twohalfmut,  # dictionary with model parameters
                  ode_with_circuit,  # ODE function for the cell with synthetic circuit
                  cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
                  # initial condition VECTOR
                  len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                  cellmodel_auxil.synth_gene_params_for_jax(par_twohalfmut, circuit_genes),
                  # synthetic gene parameters for calculating k values
                  tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
                  rtol,
                  atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
    ts = np.array(sol.ts)
    xs = np.array(sol.ys)
    # record growth rate
    _, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_twohalfmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                           circuit_eff_m_het_div_k_het)
    ls_add_twohalfmut[kplus_cntr]=np.float64(ls[-1])
    
    # one toggle (tog11 and tog12 genes) mutated
    par_onemut=par.copy()
    par_onemut['func_tog11']=0
    par_onemut['func_tog12']=0
    sol = ode_sim(par_onemut,  # dictionary with model parameters
                  ode_with_circuit,  # ODE function for the cell with synthetic circuit
                  cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
                  # initial condition VECTOR
                  len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                  cellmodel_auxil.synth_gene_params_for_jax(par_onemut, circuit_genes),
                  # synthetic gene parameters for calculating k values
                  tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
                  rtol,
                  atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
    ts = np.array(sol.ts)
    xs = np.array(sol.ys)
    # record growth rate
    _, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_onemut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                           circuit_eff_m_het_div_k_het)
    ls_add_onemut[kplus_cntr]=np.float64(ls[-1])
    
    # one toggle and a half (tog11, tog12 and tog21 genes) mutated
    par_oneandhalfmut=par.copy()
    par_oneandhalfmut['func_tog11']=0
    par_oneandhalfmut['func_tog12']=0
    par_oneandhalfmut['func_tog21']=0
    sol = ode_sim(par_oneandhalfmut,  # dictionary with model parameters
                  ode_with_circuit,  # ODE function for the cell with synthetic circuit
                  cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
                  # initial condition VECTOR
                  len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                  cellmodel_auxil.synth_gene_params_for_jax(par_oneandhalfmut, circuit_genes),
                  # synthetic gene parameters for calculating k values
                  tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
                  rtol,
                  atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
    ts = np.array(sol.ts)
    xs = np.array(sol.ys)
    # record growth rate
    _, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_oneandhalfmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                           circuit_eff_m_het_div_k_het)
    ls_add_oneandhalfmut[kplus_cntr]=np.float64(ls[-1])
    
    # both toggles (all toggle genes) mutated
    par_allmut=par.copy()
    par_allmut['func_tog11']=0
    par_allmut['func_tog12']=0
    par_allmut['func_tog21']=0
    par_allmut['func_tog22']=0
    sol = ode_sim(par_allmut,  # dictionary with model parameters
                  ode_with_circuit,  # ODE function for the cell with synthetic circuit
                  cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
                  # initial condition VECTOR
                  len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                  cellmodel_auxil.synth_gene_params_for_jax(par_allmut, circuit_genes),
                  # synthetic gene parameters for calculating k values
                  tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
                  rtol,
                  atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
    ts = np.array(sol.ts)
    xs = np.array(sol.ys)
    # record growth rate
    _, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par_allmut, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                           circuit_eff_m_het_div_k_het)
    ls_add_allmut[kplus_cntr]=np.float64(ls[-1])

In [13]:
# PLOT GROWTH RATES
font_size='8pt' # font size for the plots

# find region where overlaps fail
fail_indices = np.where(
    np.logical_or(
        np.divide(ls_add_twohalfmut, ls_add_nomut) > 1,
        np.logical_or(
            np.logical_or(np.divide(ls_add_halfmut, ls_add_nomut) > 1,
                          np.divide(ls_add_onemut, ls_add_nomut) > 1),
            np.logical_or(np.divide(ls_add_oneandhalfmut, ls_add_nomut) > 1,
                          np.divide(ls_add_allmut, ls_add_nomut) > 1)
    )))[0]
# failure regions are on either side of the functional region as growth rates change monotonically
fail_regions=[]
for i in range(0,len(fail_indices)):
    if(i==0): # first failure index is by definition the left boundary of a new failure region
        current_fail_region_left=fail_indices[i]
    elif(fail_indices[i]-fail_indices[i-1]>1): # detecting a break between failure regions
        fail_regions.append((current_fail_region_left,fail_indices[i-1])) # record the fully traversed failure region
        current_fail_region_left=fail_indices[i] # start tracking a new failure region
    elif(i==len(fail_indices)-1): # last failure index is by definition the right boundary of a new failure region
        fail_regions.append((current_fail_region_left,fail_indices[i])) # record the fully traversed failure region

In [14]:
#
# half a toggle mutated
halfmut_fig= bkplot.figure(
    frame_width=160,
    frame_height=160,
    title='tog11 mutated',
    x_axis_label="kcatyz (cat RBS:ribosome \n dissoc. rate), 1/(nM*h)",
    y_axis_label="Rel. growth rate after mutation",
    x_range=(k_plus_cats[0], k_plus_cats[-1]),
    y_range=(0.9,1.05),
    x_axis_type="log",
    tools="box_zoom,pan,hover,reset,save"
)
halfmut_fig.output_backend = "svg"

# shade regions where essential gene co-expression fails
for fail_region in fail_regions:
    halfmut_fig.add_layout(bkmodels.BoxAnnotation(left=k_plus_cats[fail_region[0]], right=k_plus_cats[fail_region[1]], fill_color='gray', fill_alpha=0.1))

# reference at 1
halfmut_fig.add_layout(bkmodels.Span(location=1, dimension='width', line_color='black', line_width=2, line_dash='dashed'))

# Toggle switches only
halfmut_fig.line(k_plus_cats,
                l_only_halfmut / l_only_nomut * np.ones(len(k_plus_cats)),
                line_color=bkRGB(72, 209, 204), line_width=2, legend_label='Toggles only')

# Punisher
halfmut_fig.line(k_plus_cats,
                l_pun_halfmut / l_pun_nomut * np.ones(len(k_plus_cats)),
                line_color=bkRGB(255, 103, 0), line_width=2, legend_label='Punisher')

# Essential gene co-expression
halfmut_fig.line(k_plus_cats,
                np.divide(ls_add_halfmut,ls_add_nomut),
                line_color=bkRGB(222, 49, 99), line_width=2, legend_label='Co-expression')

# font and axis settings
halfmut_fig.xaxis.ticker = bkmodels.LogTicker(desired_num_ticks=5)
halfmut_fig.xaxis.axis_label_text_font_size = font_size
halfmut_fig.xaxis.major_label_text_font_size = font_size
halfmut_fig.yaxis.axis_label_text_font_size = font_size
halfmut_fig.yaxis.major_label_text_font_size = font_size
halfmut_fig.legend.label_text_font_size = font_size
halfmut_fig.title.text_font_size = font_size
halfmut_fig.legend.visible=False
halfmut_fig.legend.background_fill_alpha = 1
halfmut_fig.legend.location = "bottom_right"

bkplot.show(halfmut_fig)

In [15]:
#
# two halves of a toggle mutated
twohalfmut_fig= bkplot.figure(
    frame_width=160,
    frame_height=160,
    title='tog11 and tog21 mutated',
    x_axis_label="kcatyz (cat RBS:ribosome \n dissoc. rate), 1/(nM*h)",
    y_axis_label="Rel. growth rate after mutation",
    x_range=(k_plus_cats[0], k_plus_cats[-1]),
    y_range=(0.8,1.1),
    x_axis_type="log",
    tools="box_zoom,pan,hover,reset,save"
)
twohalfmut_fig.output_backend = "svg"

# shade regions where essential gene co-expression fails
for fail_region in fail_regions:
    twohalfmut_fig.add_layout(bkmodels.BoxAnnotation(left=k_plus_cats[fail_region[0]], right=k_plus_cats[fail_region[1]], fill_color='gray', fill_alpha=0.1))

# reference at 1
twohalfmut_fig.add_layout(bkmodels.Span(location=1, dimension='width', line_color='black', line_width=2, line_dash='dashed'))

# Toggle switches only
twohalfmut_fig.line(k_plus_cats,
                l_only_twohalfmut / l_only_nomut * np.ones(len(k_plus_cats)),
                line_color=bkRGB(72, 209, 204), line_width=2, legend_label='Toggles only')

# Punisher
twohalfmut_fig.line(k_plus_cats,
                l_pun_twohalfmut / l_pun_nomut * np.ones(len(k_plus_cats)),
                line_color=bkRGB(255, 103, 0), line_width=2, legend_label='Punisher')

# Essential gene co-expression
twohalfmut_fig.line(k_plus_cats,
                np.divide(ls_add_twohalfmut,ls_add_nomut),
                line_color=bkRGB(222, 49, 99), line_width=2, legend_label='Co-expression')

# font and axis settings
twohalfmut_fig.xaxis.ticker = bkmodels.LogTicker(desired_num_ticks=5)
twohalfmut_fig.xaxis.axis_label_text_font_size = font_size
twohalfmut_fig.xaxis.major_label_text_font_size = font_size
twohalfmut_fig.yaxis.axis_label_text_font_size = font_size
twohalfmut_fig.yaxis.major_label_text_font_size = font_size
twohalfmut_fig.legend.label_text_font_size = font_size
twohalfmut_fig.title.text_font_size = font_size
twohalfmut_fig.legend.visible=False
twohalfmut_fig.legend.background_fill_alpha = 1
twohalfmut_fig.legend.location = "bottom_right"

bkplot.show(twohalfmut_fig)

In [16]:
#
# one toggle mutated
onemut_fig= bkplot.figure(
    frame_width=160,
    frame_height=160,
    title='tog11 and tog12 mutated',
    x_axis_label="kcatyz (cat RBS:ribosome \n dissoc. rate), 1/(nM*h)",
    y_axis_label="Rel. growth rate after mutation",
    x_range=(k_plus_cats[0], k_plus_cats[-1]),
    y_range=(0.5,1.25),
    x_axis_type="log",
    tools="box_zoom,pan,hover,reset,save"
)
onemut_fig.output_backend = "svg"

# shade regions where essential gene co-expression fails
for fail_region in fail_regions:
    onemut_fig.add_layout(bkmodels.BoxAnnotation(left=k_plus_cats[fail_region[0]], right=k_plus_cats[fail_region[1]], fill_color='gray', fill_alpha=0.1))

# reference at 1
onemut_fig.add_layout(bkmodels.Span(location=1, dimension='width', line_color='black', line_width=2, line_dash='dashed'))

# Toggle switches only
onemut_fig.line(k_plus_cats,
                l_only_onemut / l_only_nomut * np.ones(len(k_plus_cats)),
                line_color=bkRGB(72, 209, 204), line_width=2, legend_label='Toggles only')

# Punisher
onemut_fig.line(k_plus_cats,
                l_pun_onemut / l_pun_nomut * np.ones(len(k_plus_cats)),
                line_color=bkRGB(255, 103, 0), line_width=2, legend_label='Punisher')

# Essential gene co-expression
onemut_fig.line(k_plus_cats,
                np.divide(ls_add_onemut,ls_add_nomut),
                line_color=bkRGB(222, 49, 99), line_width=2, legend_label='Co-expression')

# font and axis settings
onemut_fig.xaxis.ticker = bkmodels.LogTicker(desired_num_ticks=5)
onemut_fig.xaxis.axis_label_text_font_size = font_size
onemut_fig.xaxis.major_label_text_font_size = font_size
onemut_fig.yaxis.axis_label_text_font_size = font_size
onemut_fig.yaxis.major_label_text_font_size = font_size
onemut_fig.legend.label_text_font_size = font_size
onemut_fig.title.text_font_size = font_size
onemut_fig.legend.visible=False
onemut_fig.legend.background_fill_alpha = 1
onemut_fig.legend.location = "right"

bkplot.show(onemut_fig)

In [17]:
#
# one toggle and a half mutated
oneandhalfmut_fig= bkplot.figure(
    frame_width=160,
    frame_height=160,
    title='tog11, tog12 and tog21 mutated',
    x_axis_label="kcatyz (cat RBS:ribosome \n dissoc. rate), 1/(nM*h)",
    y_axis_label="Rel. growth rate after mutation",
    x_range=(k_plus_cats[0], k_plus_cats[-1]),
    y_range=(0.94,1.03),
    x_axis_type="log",
    tools="box_zoom,pan,hover,reset,save"
)
oneandhalfmut_fig.output_backend = "svg"

# shade regions where essential gene co-expression fails
for fail_region in fail_regions:
    oneandhalfmut_fig.add_layout(bkmodels.BoxAnnotation(left=k_plus_cats[fail_region[0]], right=k_plus_cats[fail_region[1]], fill_color='gray', fill_alpha=0.1))

# reference at 1
oneandhalfmut_fig.add_layout(bkmodels.Span(location=1, dimension='width', line_color='black', line_width=2, line_dash='dashed'))

# Toggle switches only
oneandhalfmut_fig.line(k_plus_cats,
                l_only_oneandhalfmut / l_only_nomut * np.ones(len(k_plus_cats)),
                line_color=bkRGB(72, 209, 204), line_width=2, legend_label='Toggles only')

# Punisher
oneandhalfmut_fig.line(k_plus_cats,
                l_pun_oneandhalfmut / l_pun_nomut * np.ones(len(k_plus_cats)),
                line_color=bkRGB(255, 103, 0), line_width=2, legend_label='Punisher')

# Essential gene co-expression
oneandhalfmut_fig.line(k_plus_cats,
                np.divide(ls_add_oneandhalfmut,ls_add_nomut),
                line_color=bkRGB(222, 49, 99), line_width=2, legend_label='Co-expression')

# font and axis settings
oneandhalfmut_fig.xaxis.ticker = bkmodels.LogTicker(desired_num_ticks=5)
oneandhalfmut_fig.xaxis.axis_label_text_font_size = font_size
oneandhalfmut_fig.xaxis.major_label_text_font_size = font_size
oneandhalfmut_fig.yaxis.axis_label_text_font_size = font_size
oneandhalfmut_fig.yaxis.major_label_text_font_size = font_size
oneandhalfmut_fig.legend.label_text_font_size = font_size
oneandhalfmut_fig.title.text_font_size = font_size
oneandhalfmut_fig.legend.visible=False
oneandhalfmut_fig.legend.background_fill_alpha = 1
oneandhalfmut_fig.legend.location = "bottom_right"

bkplot.show(oneandhalfmut_fig)

In [18]:
#
# all toggles mutated
allmut_fig= bkplot.figure(
    frame_width=160,
    frame_height=160,
    title='all toggle genes mutated',
    x_axis_label="kcatyz (cat RBS:ribosome \n dissoc. rate), 1/(nM*h)",
    y_axis_label="Rel. growth rate after mutation",
    x_range=(k_plus_cats[0], k_plus_cats[-1]),
    y_range=(0.2,1.4),
    x_axis_type="log",
    tools="box_zoom,pan,hover,reset,save"
)
allmut_fig.output_backend = "svg"

# shade regions where essential gene co-expression fails
for fail_region in fail_regions:
    allmut_fig.add_layout(bkmodels.BoxAnnotation(left=k_plus_cats[fail_region[0]], right=k_plus_cats[fail_region[1]], fill_color='gray', fill_alpha=0.1))

# reference at 1
allmut_fig.add_layout(bkmodels.Span(location=1, dimension='width', line_color='black', line_width=2, line_dash='dashed'))

# Toggle switches only
allmut_fig.line(k_plus_cats,
                l_only_allmut / l_only_nomut * np.ones(len(k_plus_cats)),
                line_color=bkRGB(72, 209, 204), line_width=2, legend_label='Toggles only')

# Punisher
allmut_fig.line(k_plus_cats,
                l_pun_allmut / l_pun_nomut * np.ones(len(k_plus_cats)),
                line_color=bkRGB(255, 103, 0), line_width=2, legend_label='Punisher')

# Essential gene co-expression
allmut_fig.line(k_plus_cats,
                np.divide(ls_add_allmut,ls_add_nomut),
                line_color=bkRGB(222, 49, 99), line_width=2, legend_label='Co-expression')

# font and axis settings
allmut_fig.xaxis.ticker = bkmodels.LogTicker(desired_num_ticks=5)
allmut_fig.xaxis.axis_label_text_font_size = font_size
allmut_fig.xaxis.major_label_text_font_size = font_size
allmut_fig.yaxis.axis_label_text_font_size = font_size
allmut_fig.yaxis.major_label_text_font_size = font_size
allmut_fig.legend.label_text_font_size = font_size
allmut_fig.title.text_font_size = font_size
# allmut_fig.legend.visible=False
allmut_fig.legend.background_fill_alpha = 1
allmut_fig.legend.location = "right"

# show plots
bkplot.show(allmut_fig)

In [19]:
# FIGURE 4B - summary of relative growth rates

mutation_combinations=['tog11 mutated','tog11 and tog21\nmutated','tog11 and tog12\nmutated','tog11, tog12 and\ntog21 mutated','all toggle\ngenes mutated']
configs=['Toggles only','Punisher','Co-expression (min)']

# fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
# years = ['Toggles only', 'Punisher', 'Co-expression (min)']

# data = {'fruits' : fruits,
#         '2015'   : [2, 1, 4, 3, 2, 4],
#         '2016'   : [5, 3, 3, 2, 4, 6],
#         '2017'   : [3, 2, 4, 4, 5, 3]}

data={'mutations': mutation_combinations,
      'Toggles only': [l_only_halfmut / l_only_nomut, l_only_twohalfmut / l_only_nomut, l_only_onemut / l_only_nomut, l_only_oneandhalfmut / l_only_nomut, l_only_allmut / l_only_nomut],
      'Punisher': [l_pun_halfmut / l_pun_nomut, l_pun_twohalfmut / l_pun_nomut, l_pun_onemut / l_pun_nomut, l_pun_oneandhalfmut / l_pun_nomut, l_pun_allmut / l_pun_nomut],
      'Co-expression (min)': [min(ls_add_halfmut/ls_add_nomut), min(ls_add_twohalfmut/ls_add_nomut), min(ls_add_onemut/ls_add_nomut), min(ls_add_oneandhalfmut/ls_add_nomut), min(ls_add_allmut/ls_add_nomut)],
      'Co-expression (max)': [max(ls_add_halfmut/ls_add_nomut), max(ls_add_twohalfmut/ls_add_nomut), max(ls_add_onemut/ls_add_nomut), max(ls_add_oneandhalfmut/ls_add_nomut), max(ls_add_allmut/ls_add_nomut)]}


source = bkmodels.ColumnDataSource(data=data)

alt_fig= bkplot.figure(
    frame_width=560,
    frame_height=240,
    x_axis_label="Relative growth rate after mutation",
    y_range=mutation_combinations,
    x_range=(0.2,1.3),
    tools="box_zoom,pan,hover,reset,save"
)
alt_fig.output_backend = "svg"

alt_fig.hbar(y=bktransform.dodge('mutations', 0.25, range=alt_fig.y_range), right='Toggles only', source=source,
       height=0.2, color=bkRGB(72, 209, 204), legend_label='Toggles only')

alt_fig.hbar(y=bktransform.dodge('mutations',  0.0,  range=alt_fig.y_range), right='Punisher', source=source,
       height=0.2, color=bkRGB(255, 103, 0), legend_label='Punisher')

alt_fig.hbar(y=bktransform.dodge('mutations',  -0.25, range=alt_fig.y_range), right='Co-expression (min)', source=source,
       height=0.2, color=bkRGB(222, 49, 99), legend_label="Co-expression (min)")

alt_fig.hbar(y=bktransform.dodge('mutations',  -0.25, range=alt_fig.y_range), left='Co-expression (min)', right='Co-expression (max)', source=source,
       height=0.2, hatch_color=bkRGB(222, 49, 99), hatch_pattern='left_diagonal_line', fill_alpha=0, line_color=bkRGB(222, 49, 99),
        legend_label="Co-expression (max)")

# add a span at 1
alt_fig.add_layout(bkmodels.Span(location=1, dimension='height', line_color='black', line_width=1.5, line_dash='dashed'))

# bar settings
alt_fig.y_range.range_padding = 0.2
alt_fig.ygrid.grid_line_color = None

# axis fonts
alt_fig.xaxis.axis_label_text_font_size = font_size
alt_fig.yaxis.axis_label_text_font_size = font_size
alt_fig.xaxis.major_label_text_font_size = font_size
alt_fig.yaxis.major_label_text_font_size = font_size

# legend settings
alt_fig.legend.location = "bottom_right"
alt_fig.legend.label_text_font_size = font_size
alt_fig.legend.background_fill_alpha = 1
alt_fig.legend.padding = 2
alt_fig.legend.spacing = 1
alt_fig.legend.margin = 3

bkplot.show(alt_fig)

In [20]:
print('Admissible k_cat+ values:')
print(k_plus_cats[fail_regions[0][1]],k_plus_cats[fail_regions[1][0]])

Admissible k_cat+ values:
0.6616818201723581 1.6298574128780599
